# PCA for Data Visualization

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler,OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn import set_config
set_config(transform_output='pandas')
pd.set_option('display.max_columns',100)




In [ ]:
df = pd.read_csv("Data/modified-Data_Cortex_Nuclear.csv")
# Dropping unique IDs
df = df.drop(columns=['MouseID'])
df.info()
df.head()



In [ ]:
possible_targets = ['Type of Mouse',"Treatment",'Training','class',]
for col in possible_targets:
    print(f'\n- {col}:')
    print(df[col].value_counts())



In [ ]:
# Define X and y
target = "Training"
X = df.drop(columns = possible_targets)
y = df[target]


In [ ]:
import missingno as msno
msno.matrix(X)


In [ ]:
# Fill in nulls with 0 and verify all nulls are addressed
X = X.fillna(0)
X.isna().sum().sum()



In [ ]:
# Instantiate Standard Scaler
scaler = StandardScaler()
# Fit & transform data.
scaled_df = scaler.fit_transform(X)
scaled_df.head()


In [ ]:
# Calculating the correlation matrix and plotting
corr = scaled_df.corr()
sns.heatmap(corr, cmap='coolwarm');



In [ ]:
# Creating a scatter_matrix with pandas
pd.plotting.scatter_matrix(scaled_df, figsize=(40,40));



In [ ]:
# randomly select features to plot
np.random.seed(42)
random_features = np.random.choice(scaled_df.columns,3)
# plot thee randomly selected features
sns.pairplot(scaled_df,  vars=random_features);



In [ ]:
# Concatenate scaled features with target
plot_df = pd.concat([scaled_df, df[target]], axis=1)
# Plot with color coding based on target
g = sns.pairplot(data=plot_df,  vars=random_features, hue='Training')
g.fig.suptitle('Visualizing Raw Features - Colored by Training', y=1.01);



In [ ]:
# Instantiate PCA to make 3 principal components
pca = PCA(n_components=3)
# Create and define the principal components
principal_components = pca.fit_transform(scaled_df)
# Preview the results
principal_components.head()



In [ ]:
# Variance explained by each PC
pca.explained_variance_ratio_



In [ ]:
# Sum of variance explained by 3 principal components
pca.explained_variance_ratio_.sum()



In [ ]:
# Concatenate principal components with target
plot_df_pca = pd.concat([principal_components, df[target]], axis=1)
# Plot with color coding based on target
g_pca = sns.pairplot(data=plot_df_pca,  vars=principal_components.columns, hue='Training')
g_pca.fig.suptitle('Visualizing First 3 PCs - Colored by Training', y=1.01);



In [ ]:
import plotly.express as px
import plotly.io as pio
# Make a 3d scatter plot with a PC on each axis and color by the target
fig = px.scatter_3d(plot_df_pca, x='pca0',y='pca1',z='pca2', width=800, height=600, color = "Training")
fig



In [ ]:
def update_scatter3d(fig):
    fig.update_traces({'marker':{'size':3}})
    fig.show(config={'scrollZoom':False})
update_scatter3d(fig)



In [ ]:
# See avaialbe templates
pio.templates



In [ ]:
# Make a 3d scatter plot with a PC on each axis and color by the target
# Change template style to plotly_dark
fig = px.scatter_3d(plot_df_pca, x='pca0',y='pca1',z='pca2', width=800, height=600, color = "Training", template = 'plotly_dark')
update_scatter3d(fig)


# PCA to Speed up Machine Learning Algorithms

## PCA for Supervised Machine Learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pd.set_option('display.max_columns',200)
pd.set_option("display.max_info_rows", 800)
pd.set_option('display.max_info_columns',800)

from sklearn import set_config
set_config(transform_output='pandas')


In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def classification_metrics(y_true, y_pred, label='',
                           output_dict=False, figsize=(8,4),
                           normalize='true', cmap='Blues',
                           colorbar=False):
    # Get the classification report
    report = classification_report(y_true, y_pred)
    ## Print header and report
    header = "-"*70
    print(header, f" Classification Metrics: {label}", header, sep='\n')
    print(report)
    ## CONFUSION MATRICES SUBPLOTS
    fig, axes = plt.subplots(ncols=2, figsize=figsize)
    # create a confusion matrix  of raw counts
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=None, cmap='gist_gray', colorbar=colorbar,
                ax = axes[0],);
    axes[0].set_title("Raw Counts")
    # create a confusion matrix with the test data
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=normalize, cmap=cmap, colorbar=colorbar,
                ax = axes[1]);
    axes[1].set_title("Normalized Confusion Matrix")
    # Adjust layout and show figure
    fig.tight_layout()
    plt.show()
    # Return dictionary of classification_report
    if output_dict==True:
        report_dict = classification_report(y_true, y_pred, output_dict=True)
        return report_dict


def evaluate_classification(model, X_train, y_train, X_test, y_test,
                         figsize=(6,4), normalize='true', output_dict = False,
                            cmap_train='Blues', cmap_test="Reds",colorbar=False):
    # Get predictions for training data
    y_train_pred = model.predict(X_train)
    # Call the helper function to obtain regression metrics for training data
    results_train = classification_metrics(y_train, y_train_pred, #verbose = verbose,
                                     output_dict=True, figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_train,
                                     label='Training Data')
    print()
    # Get predictions for test data
    y_test_pred = model.predict(X_test)
    # Call the helper function to obtain regression metrics for test data
    results_test = classification_metrics(y_test, y_test_pred, #verbose = verbose,
                                  output_dict=True,figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_test,
                                    label='Test Data' )
    if output_dict == True:
        # Store results in a dataframe if ouput_frame is True
        results_dict = {'train':results_train,
                    'test': results_test}
        return results_dict



In [ ]:
# Get Data
df = pd.read_csv('Data/pd_speech_features.csv')
df.head()


In [ ]:
df.dtypes.value_counts()


In [ ]:
df.isna().sum().sum()


In [ ]:
# Define target and cols to drop
target_col = 'class'
drop_cols = ['id']
# Define X and y
y = df[target_col].copy()
X = df.drop(columns=[target_col,*drop_cols]).copy()



In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=321)
X_train.head()



In [ ]:
# Instantiate Standard Scaler
scaler = StandardScaler()
# Fit & transform data.
X_train_tf = scaler.fit_transform(X_train)
X_test_tf = scaler.transform(X_test)



In [ ]:
# Check for class balance of target
y_train.value_counts(1)



In [ ]:
from imblearn.over_sampling import SMOTE,SMOTENC
smote = SMOTE()
X_train_sm, y_train_sm = smote.fit_resample(X_train_tf, y_train)
y_train_sm.value_counts()



In [ ]:
# Import datetime library
import datetime as dt

# Record the start time
start = dt.datetime.now()

clf = RandomForestClassifier(random_state=42)

clf.fit(X_train_sm, y_train_sm)

# Record the end time and calc duration
end = dt.datetime.now()
dur_baseline = end-start

evaluate_classification(clf, X_train_sm,y_train_sm, X_test_tf, y_test)
print(f'Training time was: {dur_baseline}')



### PCA

In [ ]:
# Instantiate & fit data using PCA
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_sm)
X_test_pca = pca.transform(X_test_tf)
X_train_pca.head()



In [ ]:
# Record the start time
start = dt.datetime.now()

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_pca, y_train_sm)

# Record the end time and calc duration
end = dt.datetime.now()
dur_pca = end-start

evaluate_classification(clf, X_train_pca,y_train_sm, X_test_pca, y_test)
print(f'Training time was: {dur_pca}')


In [ ]:
# Compare speeds before and after PCA
compare_speed = dur_baseline/dur_pca
print(f"Using PCs was {compare_speed:.2f} times faster!")



In [ ]:
# Initialize and fit PCA
pca = PCA()
pca.fit(X_train_sm, y_train)



In [ ]:
# Determine how much variance is explained by each PC
explained = pd.Series(pca.explained_variance_ratio_, name='Explained Variance Ratio')
explained



In [ ]:
# plot the cumulative sum of the percentage of explained variance for each component and those before it.
ax = explained.cumsum().plot(marker='.')
# add a line to mark .9 (or 90%) variance explained
ax.axhline(.9, color='k');



### Specifying the Explained Variance
Rather than specifying the number of components to return, an alternate method is to specify the minimum proportion of explained variance you are willing to accept. PCA will automatically reduce the number of components just enough to meet your specification.

To specify the proportion of variance, give the n_components argument a float between 0 and 1, and it will return the number of components required to explain the given variance.

The code below is an example of how to ensure that enough components are returned to explain 85% of the variance. Instead of using an integer in the n_components argument to designate the number of components, we use a decimal value to indicate the amount of variance to be explained. PCA() will then automatically use enough principal components to meet this level of explanation.

In [ ]:
# Define PCA to address 85% of the variance
pca85 = PCA(n_components=.85)
# fit and transform on training data
X_train_pca85 = pca85.fit_transform(X_train_sm)
# transform test data
X_test_pca85 = pca85.transform(X_test_tf)
# obtain the number of PCs used
pca85.n_components_



In [ ]:
# Record the start time
start = dt.datetime.now()

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_pca85, y_train_sm)

# Record the end time and calc duration
end = dt.datetime.now()
dur_pca_85 = end-start

evaluate_classification(clf, X_train_pca85,y_train_sm, X_test_pca85, y_test)
print(f'Training time was: {dur_pca_85}')



In [ ]:
compare_speed = dur_baseline/dur_pca_85
print(f"Using PC's with .85 was {compare_speed:.2f} times faster!")

